<a href="https://colab.research.google.com/github/Daffaaq/Prediction-MPL-ID-S16-metode-Logistic-Regression/blob/main/week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

# ================================================
# 1. Hardcoded Match Data (Week 1 & 2)
# ================================================

match_results = [
    # Week 1
    {"team1": "ONIC", "team2": "Dewa", "score1": 2, "score2": 0},
    {"team1": "NAVI", "team2": "EVOS", "score1": 0, "score2": 2},
    {"team1": "TLID", "team2": "GEEK", "score1": 1, "score2": 2},
    {"team1": "ONIC", "team2": "BTR", "score1": 2, "score2": 0},
    {"team1": "RRQ", "team2": "AE", "score1": 2, "score2": 1},
    {"team1": "BTR", "team2": "NAVI", "score1": 1, "score2": 2},
    {"team1": "GEEK", "team2": "RRQ", "score1": 2, "score2": 0},
    {"team1": "AE", "team2": "Dewa", "score1": 2, "score2": 0},

    # Week 2
    {"team1": "ONIC", "team2": "GEEK", "score1": 2, "score2": 0},
    {"team1": "Dewa", "team2": "NAVI", "score1": 2, "score2": 0},
    {"team1": "GEEK", "team2": "BTR", "score1": 0, "score2": 2},
    {"team1": "AE", "team2": "EVOS", "score1": 2, "score2": 1},
    {"team1": "TLID", "team2": "Dewa", "score1": 1, "score2": 2},
    {"team1": "NAVI", "team2": "AE", "score1": 2, "score2": 0},
    {"team1": "RRQ", "team2": "TLID", "score1": 2, "score2": 0},
    {"team1": "BTR", "team2": "EVOS", "score1": 2, "score2": 1},
]


# ================================================
# 2. Generate Statistik Tiap Tim
# ================================================

teams = set()
stats = {}

# Initialize stats
for match in match_results:
    teams.update([match["team1"], match["team2"]])
for team in teams:
    stats[team] = {
        "match_played": 0,
        "match_won": 0,
        "game_won": 0,
        "game_lost": 0,
        "last_results": []  # win/loss streak
    }

# Simulasikan setiap match sebagai skor 2-0 atau 2-1 (acak jika menang)
import random
random.seed(42)

for match in match_results:
    t1, t2 = match["team1"], match["team2"]
    score1, score2 = match["score1"], match["score2"]

    stats[t1]["match_played"] += 1
    stats[t2]["match_played"] += 1

    stats[t1]["game_won"] += score1
    stats[t1]["game_lost"] += score2

    stats[t2]["game_won"] += score2
    stats[t2]["game_lost"] += score1

    if score1 > score2:
        stats[t1]["match_won"] += 1
        stats[t1]["last_results"].append("W")
        stats[t2]["last_results"].append("L")
    else:
        stats[t2]["match_won"] += 1
        stats[t2]["last_results"].append("W")
        stats[t1]["last_results"].append("L")


# ================================================
# 3. Encode fitur: Tim1 - Tim2 jadi vector fitur
# ================================================

def build_feature(team1, team2):
    def get_team_vector(team):
        s = stats[team]
        match_wr = s["match_won"] / s["match_played"] if s["match_played"] else 0
        game_wr = s["game_won"] / (s["game_won"] + s["game_lost"]) if (s["game_won"] + s["game_lost"]) else 0
        streak = sum([1 if r == "W" else -1 for r in s["last_results"][-3:]])  # last 3 games
        return [match_wr, game_wr, streak]

    vec1 = get_team_vector(team1)
    vec2 = get_team_vector(team2)

    # Bisa gunakan perbedaan sebagai fitur
    diff = [a - b for a, b in zip(vec1, vec2)]
    return diff

X = []
y = []

for match in match_results:
    x = build_feature(match["team1"], match["team2"])
    winner = match["team1"] if match["score1"] > match["score2"] else match["team2"]
    label = 1 if winner == match["team1"] else 0
    X.append(x)
    y.append(label)


# ================================================
# 4. Train Model Logistic Regression
# ================================================

print("=== Hasil Match Week 1 & 2 ===")
for match in match_results:
    print(f"{match['team1']} {match['score1']}:{match['score2']} {match['team2']}")

standings_before = []
for team, s in stats.items():
    game_diff = s["game_won"] - s["game_lost"]
    standings_before.append({
        "team": team,
        "match_won": s["match_won"],
        "match_played": s["match_played"],
        "game_won": s["game_won"],
        "game_lost": s["game_lost"],
        "game_diff": game_diff,
    })

standings_before.sort(key=lambda x: (-x["match_won"], -x["game_diff"], -x["game_won"]))

print("\n=== Klasemen Setelah Week 1 & 2 ===")
df_before = pd.DataFrame(standings_before)
df_before["L"] = df_before["match_played"] - df_before["match_won"]
df_before = df_before[["team", "match_won", "L", "game_won", "game_lost", "game_diff"]]
df_before.columns = ["Team", "W", "L", "GW", "GL", "GD"]
df_before.index = range(1, len(df_before) + 1)
print(df_before.to_string())



model = LogisticRegression()
model.fit(X, y)

# ================================================
# 5. Prediksi Week 3
# ================================================

week3_matches = [
    {"team1": "NAVI", "team2": "RRQ"},
    {"team1": "EVOS", "team2": "TLID"},
    {"team1": "EVOS", "team2": "GEEK"},
    {"team1": "BTR", "team2": "AE"},
    {"team1": "RRQ", "team2": "ONIC"},
    {"team1": "Dewa", "team2": "BTR"},
    {"team1": "AE", "team2": "ONIC"},
    {"team1": "TLID", "team2": "NAVI"},
]

# ================================================
# 6. Update Statistik Berdasarkan Prediksi Week 3
# ================================================
def simulate_match(team1, team2, predicted_winner):
    loser = team2 if predicted_winner == team1 else team1
    win_score = 2
    lose_score = random.choice([0, 1])  # Simulasi skor menang (bisa 2-0 atau 2-1)

    # Update statistik
    stats[team1]["match_played"] += 1
    stats[team2]["match_played"] += 1

    stats[predicted_winner]["match_won"] += 1
    stats[predicted_winner]["game_won"] += win_score
    stats[predicted_winner]["game_lost"] += lose_score

    stats[loser]["game_won"] += lose_score
    stats[loser]["game_lost"] += win_score

    stats[predicted_winner]["last_results"].append("W")
    stats[loser]["last_results"].append("L")

    # Kembalikan skor untuk dicetak
    return (win_score, lose_score) if predicted_winner == team1 else (lose_score, win_score)



predicted_results = []

print("\n=== Prediksi Week 3 ===")
for match in week3_matches:
    x_pred = [build_feature(match["team1"], match["team2"])]
    prob = model.predict_proba(x_pred)[0][1]
    pred_winner = match["team1"] if prob >= 0.5 else match["team2"]
    score1, score2 = simulate_match(match["team1"], match["team2"], pred_winner)
    predicted_results.append({
        "Team 1": match["team1"],
        "Score": f"{score1}:{score2}",
        "Team 2": match["team2"],
        "Predicted Winner": pred_winner,
        "Probability": round(prob if pred_winner == match["team1"] else 1 - prob, 2)
    })

df_pred = pd.DataFrame(predicted_results)
print(df_pred.to_string(index=False))



# ================================================
# 7. Buat Klasemen Setelah Week 3
# ================================================
standings = []
for team, s in stats.items():
    game_diff = s["game_won"] - s["game_lost"]
    standings.append({
        "team": team,
        "match_won": s["match_won"],
        "match_played": s["match_played"],
        "game_won": s["game_won"],
        "game_lost": s["game_lost"],
        "game_diff": game_diff,
    })

# Urutkan berdasarkan aturan MPL:
# 1. Match win paling banyak
# 2. Game difference
# 3. Game won (jika masih seri)
standings.sort(key=lambda x: (-x["match_won"], -x["game_diff"], -x["game_won"]))

# Tampilkan klasemen
print("\n=== Klasemen Akhir Week 3 (Prediksi) ===")
df_after = pd.DataFrame(standings)
df_after["L"] = df_after["match_played"] - df_after["match_won"]
df_after = df_after[["team", "match_won", "L", "game_won", "game_lost", "game_diff"]]
df_after.columns = ["Team", "W", "L", "GW", "GL", "GD"]
df_after.index = range(1, len(df_after) + 1)
print(df_after.to_string())


# ================================================
# 8. Masukkan Hasil Asli Week 3
# ================================================

actual_week3_results = [
    {"team1": "NAVI", "team2": "RRQ", "score1": 0, "score2": 2},
    {"team1": "EVOS", "team2": "TLID", "score1": 2, "score2": 0},
    {"team1": "EVOS", "team2": "GEEK", "score1": 2, "score2": 0},
    {"team1": "BTR", "team2": "AE", "score1": 2, "score2": 1},
    {"team1": "RRQ", "team2": "ONIC", "score1": 0, "score2": 2},
    {"team1": "Dewa", "team2": "BTR", "score1": 1, "score2": 2},
    {"team1": "AE", "team2": "ONIC", "score1": 1, "score2": 2},
    {"team1": "TLID", "team2": "NAVI", "score1": 0, "score2": 2},
]

def update_stats_from_actual(matches):
    for match in matches:
        t1, t2 = match["team1"], match["team2"]
        s1, s2 = match["score1"], match["score2"]

        stats[t1]["match_played"] += 1
        stats[t2]["match_played"] += 1

        stats[t1]["game_won"] += s1
        stats[t1]["game_lost"] += s2
        stats[t2]["game_won"] += s2
        stats[t2]["game_lost"] += s1

        if s1 > s2:
            stats[t1]["match_won"] += 1
            stats[t1]["last_results"].append("W")
            stats[t2]["last_results"].append("L")
        else:
            stats[t2]["match_won"] += 1
            stats[t2]["last_results"].append("W")
            stats[t1]["last_results"].append("L")

# Masukkan hasil asli week 3 ke dalam stats
update_stats_from_actual(actual_week3_results)

# ================================================
# 9. Prediksi Week 4
# ================================================

week4_matches = [
    {"team1": "NAVI", "team2": "ONIC"},
    {"team1": "EVOS", "team2": "Dewa"},
    {"team1": "TLID", "team2": "BTR"},
    {"team1": "RRQ", "team2": "EVOS"},
    {"team1": "GEEK", "team2": "AE"},
    {"team1": "ONIC", "team2": "TLID"},
    {"team1": "BTR", "team2": "RRQ"},
    {"team1": "Dewa", "team2": "GEEK"},
]

predicted_week4 = []

print("\n=== Prediksi Week 4 ===")
for match in week4_matches:
    x_pred = [build_feature(match["team1"], match["team2"])]
    prob = model.predict_proba(x_pred)[0][1]
    pred_winner = match["team1"] if prob >= 0.5 else match["team2"]
    score1, score2 = simulate_match(match["team1"], match["team2"], pred_winner)
    predicted_week4.append({
        "Team 1": match["team1"],
        "Score": f"{score1}:{score2}",
        "Team 2": match["team2"],
        "Predicted Winner": pred_winner,
        "Probability": round(prob if pred_winner == match["team1"] else 1 - prob, 2)
    })

df_pred4 = pd.DataFrame(predicted_week4)
print(df_pred4.to_string(index=False))


# ================================================
# 10. Klasemen Setelah Week 4 (Prediksi)
# ================================================

final_standings = []
for team, s in stats.items():
    game_diff = s["game_won"] - s["game_lost"]
    final_standings.append({
        "team": team,
        "match_won": s["match_won"],
        "match_played": s["match_played"],
        "game_won": s["game_won"],
        "game_lost": s["game_lost"],
        "game_diff": game_diff,
    })

final_standings.sort(key=lambda x: (-x["match_won"], -x["game_diff"], -x["game_won"]))

print("\n=== Klasemen Akhir Week 4 (Prediksi) ===")
df_final = pd.DataFrame(final_standings)
df_final["L"] = df_final["match_played"] - df_final["match_won"]
df_final = df_final[["team", "match_won", "L", "game_won", "game_lost", "game_diff"]]
df_final.columns = ["Team", "W", "L", "GW", "GL", "GD"]
df_final.index = range(1, len(df_final) + 1)
print(df_final.to_string())


# Week 4 actual results
actual_week4_results = [
    {"team1": "NAVI", "team2": "ONIC", "score1": 1, "score2": 2},
    {"team1": "EVOS", "team2": "Dewa", "score1": 2, "score2": 0},
    {"team1": "TLID", "team2": "BTR", "score1": 0, "score2": 2},
    {"team1": "RRQ", "team2": "EVOS", "score1": 2, "score2": 0},
    {"team1": "GEEK", "team2": "AE", "score1": 0, "score2": 2},
    {"team1": "ONIC", "team2": "TLID", "score1": 2, "score2": 0},
    {"team1": "BTR", "team2": "RRQ", "score1": 2, "score2": 1},
    {"team1": "Dewa", "team2": "GEEK", "score1": 2, "score2": 1},
]

# Update stats
update_stats_from_actual(actual_week4_results)


week5_matches = [
    {"team1": "GEEK", "team2": "NAVI"},
    {"team1": "EVOS", "team2": "ONIC"},
    {"team1": "Dewa", "team2": "RRQ"},
    {"team1": "AE", "team2": "TLID"},
    {"team1": "EVOS", "team2": "BTR"},
    {"team1": "AE", "team2": "NAVI"},
    {"team1": "GEEK", "team2": "ONIC"},
    {"team1": "Dewa", "team2": "TLID"},
]

predicted_week5 = []

print("\n=== Prediksi Week 5 ===")
for match in week5_matches:
    x_pred = [build_feature(match["team1"], match["team2"])]
    prob = model.predict_proba(x_pred)[0][1]
    pred_winner = match["team1"] if prob >= 0.5 else match["team2"]
    score1, score2 = simulate_match(match["team1"], match["team2"], pred_winner)
    predicted_week5.append({
        "Team 1": match["team1"],
        "Score": f"{score1}:{score2}",
        "Team 2": match["team2"],
        "Predicted Winner": pred_winner,
        "Probability": round(prob if pred_winner == match["team1"] else 1 - prob, 2)
    })

df_pred5 = pd.DataFrame(predicted_week5)
print(df_pred5.to_string(index=False))


final_standings_week5 = []
for team, s in stats.items():
    game_diff = s["game_won"] - s["game_lost"]
    final_standings_week5.append({
        "team": team,
        "match_won": s["match_won"],
        "match_played": s["match_played"],
        "game_won": s["game_won"],
        "game_lost": s["game_lost"],
        "game_diff": game_diff,
    })

# Urutkan sesuai aturan MPL
final_standings_week5.sort(key=lambda x: (-x["match_won"], -x["game_diff"], -x["game_won"]))

print("\n=== Klasemen Akhir Week 5 (Prediksi) ===")
df_final5 = pd.DataFrame(final_standings_week5)
df_final5["L"] = df_final5["match_played"] - df_final5["match_won"]
df_final5 = df_final5[["team", "match_won", "L", "game_won", "game_lost", "game_diff"]]
df_final5.columns = ["Team", "W", "L", "GW", "GL", "GD"]
df_final5.index = range(1, len(df_final5) + 1)
print(df_final5.to_string())




=== Hasil Match Week 1 & 2 ===
ONIC 2:0 Dewa
NAVI 0:2 EVOS
TLID 1:2 GEEK
ONIC 2:0 BTR
RRQ 2:1 AE
BTR 1:2 NAVI
GEEK 2:0 RRQ
AE 2:0 Dewa
ONIC 2:0 GEEK
Dewa 2:0 NAVI
GEEK 0:2 BTR
AE 2:1 EVOS
TLID 1:2 Dewa
NAVI 2:0 AE
RRQ 2:0 TLID
BTR 2:1 EVOS

=== Klasemen Setelah Week 1 & 2 ===
   Team  W  L  GW  GL  GD
1  ONIC  3  0   6   0   6
2   RRQ  2  1   4   3   1
3   BTR  2  2   5   5   0
4    AE  2  2   5   5   0
5  Dewa  2  2   4   5  -1
6  GEEK  2  2   4   5  -1
7  NAVI  2  2   4   5  -1
8  EVOS  1  2   4   4   0
9  TLID  0  3   2   6  -4

=== Prediksi Week 3 ===
Team 1 Score Team 2 Predicted Winner  Probability
  NAVI   2:0    RRQ             NAVI         0.66
  EVOS   2:0   TLID             EVOS         0.88
  EVOS   2:1   GEEK             EVOS         0.68
   BTR   2:0     AE              BTR         0.67
   RRQ   0:2   ONIC             ONIC         0.85
  Dewa   0:2    BTR              BTR         0.61
    AE   0:2   ONIC             ONIC         0.85
  TLID   0:2   NAVI             NAVI  

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd

# ================================================
# 1. Hardcoded Match Data Week 1-4 (hasil asli)
# ================================================
match_results = [
    # Week 1
    {"team1": "ONIC", "team2": "Dewa", "score1": 2, "score2": 0},
    {"team1": "NAVI", "team2": "EVOS", "score1": 0, "score2": 2},
    {"team1": "TLID", "team2": "GEEK", "score1": 1, "score2": 2},
    {"team1": "ONIC", "team2": "BTR", "score1": 2, "score2": 0},
    {"team1": "RRQ", "team2": "AE", "score1": 2, "score2": 1},
    {"team1": "BTR", "team2": "NAVI", "score1": 1, "score2": 2},
    {"team1": "GEEK", "team2": "RRQ", "score1": 2, "score2": 0},
    {"team1": "AE", "team2": "Dewa", "score1": 2, "score2": 0},

    # Week 2
    {"team1": "ONIC", "team2": "GEEK", "score1": 2, "score2": 0},
    {"team1": "Dewa", "team2": "NAVI", "score1": 2, "score2": 0},
    {"team1": "GEEK", "team2": "BTR", "score1": 0, "score2": 2},
    {"team1": "AE", "team2": "EVOS", "score1": 2, "score2": 1},
    {"team1": "TLID", "team2": "Dewa", "score1": 1, "score2": 2},
    {"team1": "NAVI", "team2": "AE", "score1": 2, "score2": 0},
    {"team1": "RRQ", "team2": "TLID", "score1": 2, "score2": 0},
    {"team1": "BTR", "team2": "EVOS", "score1": 2, "score2": 1},

    # Week 3
    {"team1": "NAVI", "team2": "RRQ", "score1": 0, "score2": 2},
    {"team1": "EVOS", "team2": "TLID", "score1": 2, "score2": 0},
    {"team1": "EVOS", "team2": "GEEK", "score1": 2, "score2": 0},
    {"team1": "BTR", "team2": "AE", "score1": 2, "score2": 1},
    {"team1": "RRQ", "team2": "ONIC", "score1": 0, "score2": 2},
    {"team1": "Dewa", "team2": "BTR", "score1": 1, "score2": 2},
    {"team1": "AE", "team2": "ONIC", "score1": 1, "score2": 2},
    {"team1": "TLID", "team2": "NAVI", "score1": 0, "score2": 2},

    # Week 4
    {"team1": "NAVI", "team2": "ONIC", "score1": 1, "score2": 2},
    {"team1": "EVOS", "team2": "Dewa", "score1": 2, "score2": 0},
    {"team1": "TLID", "team2": "BTR", "score1": 0, "score2": 2},
    {"team1": "RRQ", "team2": "EVOS", "score1": 2, "score2": 0},
    {"team1": "GEEK", "team2": "AE", "score1": 0, "score2": 2},
    {"team1": "ONIC", "team2": "TLID", "score1": 2, "score2": 0},
    {"team1": "BTR", "team2": "RRQ", "score1": 2, "score2": 1},
    {"team1": "Dewa", "team2": "GEEK", "score1": 2, "score2": 1},
]

# ================================================
# 2. Hitung Statistik Tim dari Data Asli
# ================================================
teams = set()
for m in match_results:
    teams.update([m["team1"], m["team2"]])

def init_stats():
    return {team: {"match_played": 0, "match_won": 0,
                   "game_won": 0, "game_lost": 0,
                   "last_results": []} for team in teams}

def update_stats(matches, stats):
    for match in matches:
        t1, t2, s1, s2 = match["team1"], match["team2"], match["score1"], match["score2"]
        stats[t1]["match_played"] += 1
        stats[t2]["match_played"] += 1
        stats[t1]["game_won"] += s1
        stats[t1]["game_lost"] += s2
        stats[t2]["game_won"] += s2
        stats[t2]["game_lost"] += s1
        if s1 > s2:
            stats[t1]["match_won"] += 1
            stats[t1]["last_results"].append("W")
            stats[t2]["last_results"].append("L")
        else:
            stats[t2]["match_won"] += 1
            stats[t2]["last_results"].append("W")
            stats[t1]["last_results"].append("L")

# ================================================
# 3. Feature Builder
# ================================================
def build_feature(team1, team2, stats):
    def vec(team):
        s = stats[team]
        match_wr = s["match_won"]/s["match_played"] if s["match_played"] else 0
        game_wr = s["game_won"]/(s["game_won"]+s["game_lost"]) if (s["game_won"]+s["game_lost"]) else 0
        streak = sum([1 if r=="W" else -1 for r in s["last_results"][-3:]])
        return [match_wr, game_wr, streak]
    v1, v2 = vec(team1), vec(team2)
    return [a-b for a,b in zip(v1,v2)]

# ================================================
# 4. Data Training (Multi-class Skor)
# ================================================
stats_actual = init_stats()
update_stats(match_results, stats_actual)

X, y = [], []
for m in match_results:
    feat = build_feature(m["team1"], m["team2"], stats_actual)
    X.append(feat)
    y.append(f"{m['score1']}-{m['score2']}")

model = LogisticRegression(max_iter=2000)
model.fit(X, y)

# ================================================
# 5. Prediksi dengan Probabilitas Skor
# ================================================
def predict_match(team1, team2, topn=4):
    feat = [build_feature(team1, team2, stats_actual)]
    probs = model.predict_proba(feat)[0]
    classes = model.classes_
    df = pd.DataFrame({"Score": classes, "Prob": probs})
    df = df.sort_values("Prob", ascending=False).head(topn)
    return df.reset_index(drop=True)

def predict_week(matches, topn=4):
    preds = []
    for m in matches:
        df = predict_match(m["team1"], m["team2"], topn=topn)
        preds.append({
            "Match": f"{m['team1']} vs {m['team2']}",
            "Predictions": ", ".join([f"{r.Score} ({r.Prob:.2%})" for _,r in df.iterrows()])
        })
    return pd.DataFrame(preds)


# ================================================
# 6. Generate Klasemen Setelah Prediksi
# ================================================
def generate_standings(all_matches):
    stats_pred = init_stats()
    update_stats(all_matches, stats_pred)
    table = []
    for t,s in stats_pred.items():
        gd = s["game_won"] - s["game_lost"]
        table.append({
            "Team": t,
            "W": s["match_won"],
            "L": s["match_played"]-s["match_won"],
            "GW": s["game_won"],
            "GL": s["game_lost"],
            "GD": gd
        })
    df = pd.DataFrame(table)
    df = df.sort_values(by=["W","GD","GW"], ascending=[False,False,False]).reset_index(drop=True)
    df.index = range(1,len(df)+1)
    return df

# ================================================
# 7. Prediksi Week 5
# ================================================
week5_matches = [
    {"team1": "GEEK", "team2": "NAVI"},
    {"team1": "EVOS", "team2": "ONIC"},
    {"team1": "Dewa", "team2": "RRQ"},
    {"team1": "AE", "team2": "TLID"},
    {"team1": "EVOS", "team2": "BTR"},
    {"team1": "AE", "team2": "NAVI"},
    {"team1": "GEEK", "team2": "ONIC"},
    {"team1": "Dewa", "team2": "TLID"},
]

print("\n=== Prediksi Week 5 (Probabilitas Semua Skor) ===")
print(predict_week(week5_matches, topn=4).to_string(index=False))

# Untuk klasemen, ambil skor prediksi paling mungkin (argmax)
predicted_matches = []
for m in week5_matches:
    df = predict_match(m["team1"], m["team2"], topn=1)
    score = df.loc[0,"Score"]
    s1,s2 = map(int, score.split("-"))
    predicted_matches.append({"team1": m["team1"], "team2": m["team2"], "score1": s1, "score2": s2})

all_matches = match_results + predicted_matches
print("\n=== Klasemen Setelah Prediksi Week 5 ===")
print(generate_standings(all_matches).to_string())



=== Prediksi Week 5 (Probabilitas Semua Skor) ===
       Match                                            Predictions
GEEK vs NAVI 0-2 (31.77%), 1-2 (28.64%), 2-0 (27.44%), 2-1 (12.15%)
EVOS vs ONIC 1-2 (35.04%), 0-2 (32.85%), 2-0 (19.19%), 2-1 (12.91%)
 Dewa vs RRQ 2-0 (36.90%), 1-2 (24.81%), 2-1 (24.45%), 0-2 (13.85%)
  AE vs TLID   2-0 (70.28%), 2-1 (20.17%), 1-2 (6.75%), 0-2 (2.80%)
 EVOS vs BTR 0-2 (30.90%), 2-0 (29.91%), 1-2 (27.72%), 2-1 (11.47%)
  AE vs NAVI 2-0 (47.19%), 2-1 (20.73%), 1-2 (19.76%), 0-2 (12.32%)
GEEK vs ONIC   0-2 (72.18%), 1-2 (24.07%), 2-0 (2.30%), 2-1 (1.45%)
Dewa vs TLID   2-0 (67.82%), 2-1 (21.90%), 1-2 (7.30%), 0-2 (2.98%)

=== Klasemen Setelah Prediksi Week 5 ===
   Team  W  L  GW  GL  GD
1  ONIC  9  0  18   3  15
2   BTR  7  2  15   8   7
3    AE  5  4  13   9   4
4  Dewa  5  4  11  10   1
5  EVOS  4  5  11  10   1
6   RRQ  4  4   9   9   0
7  NAVI  4  5   9  11  -2
8  GEEK  2  7   5  15 -10
9  TLID  0  9   2  18 -16
